# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [9]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key
from holoviews import dim, opts

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")


In [10]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bisho,-32.8472,27.4422,291.75,93,100,1.46,ZA,1669851213
1,1,rikitea,-23.1203,-134.9692,296.00,71,82,5.76,PF,1669851213
2,2,eruwa,7.5333,3.4167,297.67,81,52,0.89,NG,1669851214
3,3,copiapo,-27.3667,-70.3333,288.98,62,1,2.64,CL,1669850967
4,4,dikson,73.5069,80.5464,267.85,95,100,5.42,RU,1669851214


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [11]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

humidity_data = city_data_df.hvplot.points('Lng', 'Lat', color='City', geo=True,
                     tiles='OSM', size='Humidity', scale=1,
                     frame_width=800, frame_height=800)# YOUR CODE HERE
humidity_data
# Display the map plot


# Display the map
# YOUR CODE HERE

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [12]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE

ideal_weather=city_data_df.loc[(city_data_df["Max Temp"]<= 300)&(city_data_df["Max Temp"]>= 150) & ( city_data_df['Wind Speed'] <=4.5) & (city_data_df["Cloudiness"] == 0), :]
ideal_weather
# Drop any rows with null values
# YOUR CODE HERE
ideal_weather=ideal_weather.dropna()
# Display sample data
# YOUR CODE HERE
ideal_weather.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
10,10,provideniya,64.3833,-173.3000,271.21,74,0,3.00,RU,1669851218
31,31,shahpura,27.3833,75.9667,286.32,52,0,0.86,IN,1669851227
32,32,pokhara,28.2333,83.9833,287.49,36,0,2.26,NP,1669850949
65,65,krasnyy yar,46.5331,48.3456,266.11,33,0,3.80,RU,1669851241
93,93,shunyi,40.1250,116.6453,259.12,72,0,0.00,CN,1669851271


### Step 3: Create a new DataFrame called `hotel_df`.

In [33]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df=ideal_weather.copy()
hotel_df=hotel_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']]
hotel_df['Hotel Name']= ''
#hotel_df
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE

# Display sample data
# YOUR CODE HERE

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [34]:
# Set parameters to search for a hotel

radius = 10000 # YOUR CODE HERE

params = {"apiKey":geoapify_key,
          "format":"json",
          "categories":"accommodation.hotel"
         
         } 


# Set the base URL
base_url = "https://api.geoapify.com/v1/geocode/search"
# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    longitude = hotel_df.loc[index, 'Lng']
    latitude = hotel_df.loc[index, 'Lat']
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params).json()
    
    # Convert the API response to JSON format
    
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
   
    
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data



Starting hotel search
provideniya - nearest hotel: No hotel found
shahpura - nearest hotel: Alila Fort Bishangarh
pokhara - nearest hotel: Hotel Annapurna View Sarangkot
krasnyy yar - nearest hotel: Веранда
shunyi - nearest hotel: 顺义宾馆
port elizabeth - nearest hotel: Waterford Hotel
laguna - nearest hotel: Holiday Inn Express & Suites
lalsot - nearest hotel: No hotel found
carlyle - nearest hotel: No hotel found
clyde river - nearest hotel: No hotel found
springfield - nearest hotel: Hotel Vandivort
aswan - nearest hotel: Yaseen hotel
pacific grove - nearest hotel: Pacific Grove Inn
waddan - nearest hotel: No hotel found
saint george - nearest hotel: The Advenire, an Autograph Collection Hotel
batesville - nearest hotel: No hotel found
lincoln - nearest hotel: New Victorian Suites Lincoln
kodiak - nearest hotel: Shelikof Lodge
semnan - nearest hotel: مرکز آموزشی رفاهی فرهنگیان - خانه معلم
sorrento - nearest hotel: Rivoli
taoudenni - nearest hotel: No hotel found
ulaanbaatar - nearest h

In [35]:
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
10,provideniya,RU,64.3833,-173.3000,74,No hotel found
31,shahpura,IN,27.3833,75.9667,52,Alila Fort Bishangarh
32,pokhara,NP,28.2333,83.9833,36,Hotel Annapurna View Sarangkot
65,krasnyy yar,RU,46.5331,48.3456,33,Веранда
93,shunyi,CN,40.1250,116.6453,72,顺义宾馆
100,port elizabeth,ZA,-33.9180,25.5701,90,Waterford Hotel
103,laguna,US,38.4210,-121.4238,47,Holiday Inn Express & Suites
104,lalsot,IN,26.5667,76.3333,53,No hotel found
129,carlyle,US,38.6103,-89.3726,56,No hotel found
140,clyde river,CA,70.4692,-68.5914,68,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [40]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
hotel_in_ideal_weather = hotel_df.hvplot.points('Lng', 'Lat', color='City', geo=True,
                     tiles='OSM',  hover_cols=['Hotel Name', 'Country'],size='Humidity', scale=1,
                     frame_width=800, frame_height=800)# YOUR CODE HERE
hotel_in_ideal_weather
# Display the 

# Display the map
# YOUR CODE HERE

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)